## 1. Initialize the project

Create the working context: data download project if not created already. Project is a placeholder for the code, data, and management of the data operations.

In [ ]:
import digitalhub as dh
PROJECT_NAME = "<YOUR_PROJECT_NAME>"
proj = dh.get_or_create_project(PROJECT_NAME)

## 2. Prepare data

This section describes the required artifacts (sentinel data, shape and map files) needed to run the elaboration for the requested area of interest.

### 1- Sentinel data

Please note that this function 'elaborate-geological-data' depends on Sentinel data artifacts, which must be logged in to the project context(if not already) both for ascending and descending orbits using the catalog function 'download-sentinel-data'. For more detailed information, please refer to the catalog function [download-sentinel-data](../download-sentinel-data/README.md)

### 2- Shape and Map data.
Save the necessary artifacts (shape, map files) for the required area of interest inside to the project context.  For e.g. shape file for Trentino region can be downloaded from the open data portal link below.

https://siatservices.provincia.tn.it/idt/vector/p_TN_377793f1-1094-4e81-810e-403897418b23.zip. 

Unzip the files in a folder named 'Shapes_AOI' and then log it in project context.

In [ ]:
artifact_name='Shapes_AOI'
src_path='/Shapes_AOI'
artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

Log the Map aritfact with three files (slope map, aspect map, and legend.qml). For e.g trentino_slope_map.tiff, trentino_aspect_map.tiff, and legend.qml can be downloaded from the Huggingface repository.

https://huggingface.co/datasets/lbergamasco/trentino-slope-map/tree/main

Copy the three files inside a folder 'Map' and log it as project artifact

In [ ]:
artifact_name='Map'
src_path='Map'
artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

The resulting dataset will be registered as the project artifacts in the datalake under the name 'Shapes_AOI' and 'Map'.


## 3. Execution

Fetch the "elaborate-geological-data" operation in the project. 

In [ ]:
function_el = proj.get_function("elaborate-geological-data") 

Run the function as shown below. 

In [ ]:
run_el = function_el.run(
    action="job",
    fs_group='8877',
    resources={
        "cpu": {"requests": "6", "limits": "12"},
        "mem": {"requests": "32Gi", "limits": "64Gi"}
    },
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-land",
        "mount_path": "/app/files",
        "spec": { "size": "600Gi" }
    }],
    args=[
        '/shared/launch.sh',
        's1_ascending',
        's1_descending',
        '2021-03-01',
        '2021-07-30',
        'landslide_2020-11-01_2020-11-14',
        'Shapes_AOI',
        'ammprv_v.shp',
        'Map',
        'POLYGON ((10.595369 45.923394, 10.644894 45.923394, 10.644894 45.945838, \
                   10.595369 45.945838, 10.595369 45.923394))'
    ]
)